# **EMOTION DETECTION**

In [63]:
# Getting all dependencies

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import pickle


In [64]:
# Getting files on Collab
# uploaded = files.upload()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [65]:
# Specify the name of the uploaded zip file
# zip_file_name = 'images.zip'

# Unzip the file
# with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    # zip_ref.extractall('')

# Verify the contents
# print(os.listdir('images'))

In [66]:
# Setting up the directories

TRAINING_DIR = './images/train'
TEST_DIR = './images/validation'

In [67]:
# Making dataframes for the training and test data (with images and labels)

def create_dataframe(directory):
    image_paths = []
    labels = []

    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image_paths.append(image_path)
                labels.append(label)
            print(f"Completed {label_dir}")
    return image_paths, labels

In [68]:
# Making Train and Test dataframes
train_df = pd.DataFrame()
train_df['image_path'], train_df['label'] = create_dataframe(TRAINING_DIR)

test_df = pd.DataFrame()
test_df['image_path'], test_df['label'] = create_dataframe(TEST_DIR)
print("Train and test dataframes created successfully.")

print(train_df)
print(test_df)

Completed ./images/train\angry
Completed ./images/train\disgust
Completed ./images/train\fear
Completed ./images/train\happy
Completed ./images/train\neutral
Completed ./images/train\sad
Completed ./images/train\surprise
Completed ./images/validation\angry
Completed ./images/validation\disgust
Completed ./images/validation\fear
Completed ./images/validation\happy
Completed ./images/validation\neutral
Completed ./images/validation\sad
Completed ./images/validation\surprise
Train and test dataframes created successfully.
                             image_path     label
0            ./images/train\angry\0.jpg     angry
1            ./images/train\angry\1.jpg     angry
2           ./images/train\angry\10.jpg     angry
3        ./images/train\angry\10002.jpg     angry
4        ./images/train\angry\10016.jpg     angry
...                                 ...       ...
29208  ./images/train\surprise\9969.jpg  surprise
29209  ./images/train\surprise\9985.jpg  surprise
29210  ./images/train\sur

In [69]:
# Label Encoding

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
y_train = label_encoder.transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

In [70]:
# Training Transform (with augmentation)
train_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
# Test Transform (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize((48, 48)),  # Resize to 48x48
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

In [71]:
# Custom Dataset class

class EmotionDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('L')
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

In [72]:
# Create datasets and dataloaders

train_dataset = EmotionDataset(train_df['image_path'].tolist(), y_train, transform=train_transform)
test_dataset = EmotionDataset(test_df['image_path'].tolist(), y_test, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [73]:
# Convolutional Neural Network (CNN) model
# The model is a sequential model with convolutional layers, max pooling layers, and dropout layers
# The model is designed to classify images into 7 different classes (emotions)

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.1),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.1),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.1),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.1),

            # ✅ Added Conv layer: 256 → 512
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.1)
        )

        # Output feature map size is now (512, 1, 1) assuming input is 48x48 (FER dataset)
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 1 * 1, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 7)  # Directly output 7 classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [74]:
# Configuring the model
# from sklearn.utils.class_weight import compute_class_weight

# # Compute class weights
# class_weights = compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(train_df['label']),
#     y=train_df['label']
# )
# class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# # Use weights in loss function
# criterion = nn.CrossEntropyLoss(weight=class_weights)
model = EmotionCNN().to(device)
# model.load_state_dict(torch.load("emotionDetector.pth", map_location=device))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [75]:
# Training Model

def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=150, save_path='best_model.pth'):
    best_val_acc = 0.0  # Track the best validation accuracy

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device).long()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_acc = 100 * train_correct / train_total
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device).long()
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        val_loss_avg = val_loss / len(test_loader)
        print(f"Validation Loss: {val_loss_avg:.4f}, Validation Accuracy: {val_acc:.2f}%")
        
        # Save model if it has the best validation accuracy so far
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(f"✅ New best model saved with val acc: {val_acc:.2f}%")

# Example usage
train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=150,
    save_path='emotionDetector_ClassChange128_transformationChange.pth'
)

100%|██████████| 229/229 [00:24<00:00,  9.40it/s]


Epoch [1/150], Train Loss: 1.7075, Train Accuracy: 30.85%
Validation Loss: 1.4415, Validation Accuracy: 44.23%
✅ New best model saved with val acc: 44.23%


100%|██████████| 229/229 [00:25<00:00,  8.84it/s]


Epoch [2/150], Train Loss: 1.4423, Train Accuracy: 44.28%
Validation Loss: 1.3619, Validation Accuracy: 48.26%
✅ New best model saved with val acc: 48.26%


100%|██████████| 229/229 [00:26<00:00,  8.60it/s]


...

Epoch [148/150], Train Loss: 0.5277, Train Accuracy: 80.69%
Validation Loss: 0.9948, Validation Accuracy: 70.03%


100%|██████████| 229/229 [00:35<00:00,  6.38it/s]


Epoch [149/150], Train Loss: 0.5183, Train Accuracy: 80.80%
Validation Loss: 0.9940, Validation Accuracy: 70.21%


100%|██████████| 229/229 [00:36<00:00,  6.31it/s]


Epoch [150/150], Train Loss: 0.5170, Train Accuracy: 80.96%
Validation Loss: 1.0015, Validation Accuracy: 69.91%


In [76]:
# Save model
# torch.save(model.state_dict(), "emotionDetector_67_150epoch.pth")
print("Model saved successfully.")

Model saved successfully.


In [77]:
# For Collab
# Download the JSON file
# files.download("emotionDetector.json")

# Download the HDF5 file
# files.download("emotionDetector.h5")